# Working with obstacles
This notebook aims at giving the basis to work with obstacles, first by examplifying the basic API of HPP-FCL, then by writing an optimization problem under noncollision avoidance constraint. One of the main message is that the collision constraint is hard, and that a lot of work remains necessary, to make it efficient in practice.

We first show how to compute the minimal ellipsoid that encapsulate a body of the robot. 
Then a inverse-geometry problem is written under the constraint that a set of points remains outside of the encapsulating ellipsoid.

The notebook is written for a simple UR10 robot, and you should be able to mix it with notebook #3 to make a complete trajectory optimization for a humanoid.



In [ ]:
import magic_donotload

## Set up
We will use the following tools:
- the ur10 model and the Talos model (loaded by example-robot-data)
- HPP-FCL, through the Pinocchio API
- pinocchio.casadi for writing the problem and computing its derivatives
- the IpOpt solver wrapped in casadi
- the meshcat viewer

In [ ]:
# %load tp4/generated/encapsulating_ellipses_import
import pinocchio as pin
from pinocchio import casadi as cpin
import casadi
import numpy as np
import example_robot_data as robex
from utils.meshcat_viewer_wrapper import MeshcatVisualizer


We will load the UR10 model, but feel free to change it for any model you like.

In [ ]:
# %load tp4/generated/encapsulating_ellipses_load
# --- Load robot model
robot = robex.load('ur10')
viz = MeshcatVisualizer(robot)
viz.display(robot.q0)


In [ ]:
viz.viewer.jupyter_cell()

## Accessing the vertices of the collisions objects
Let's quickly summarize the basics of collision avoidance in Pinocchio. Our models are most often loaded from URDF storage, which typically contains two sets of 3D objects: the visuals, very detailed and often colored meshed used for display; and the collisions, less detailed objects used for a fair computation of the collisions. Both are stored in the robot_wrapper, under collision_model and visual_model.

In [ ]:
print(robot.collision_model,robot.visual_model)


Both are corresponding to the same structure of type pin.GeometryModel. It mostly contains a list of geometry objects (that we are going to use) and a list of collision pair (very useful, but not in this notebook).


In [ ]:
geom=robot.collision_model.geometryObjects[0]

A geometry object is a 3d shape attached to a parent joint at a specific placement. It mostly contains 3 fields: geom.parentJoint the index of the parent joint ; geom.placement, the placement of the 3d shape in the frame of the parent joint; and geom.geometry, a specific structure representing the shape.

In [ ]:
shape = geom.geometry
print(shape)

Shapes can be simple features (sphere, capsule, ellipsoid, box, etc), or soup of polygon. This is the most frequent case when loading a model from URDF. Then we are mostly interested by the list of vertices:

In [ ]:
vertices = shape.vertices()
print(vertices)

Most of the time, when you are working with HPP-FCL through Pinocchio, you will not go that deep in the collision description, but rather access it through high level functions of HPP-FCL, like computing distances, detecting collisions, etc. Here we are going to use this list to compute the minimal encapsulating ellispoid.

## Encapsulating

### Problem formulation
An ellispoid $\mathcal{E}$ can be defined by a symmetric positive matrix $A$ and a center 3d vector $c$, as the set of points respecting:
$$ \forall p \in \mathcal{E}, (p-c)^T A (p-c) \le 1$$

The matrix $A$ can be deduced from the orientation of the main axes $R$ and the 3 radii $r=(r_1,r_2,r_3)$ by:
$$A=R \ \textrm{diag}(\frac{1}{r^2}) R^T$$
where $\textrm{diag}(\frac{1}{r^2})$ is the diagonal matrix formed with the inverted squared radii.

Inside an optimization program, $R$ is more conveniently represented by ... (if you thought Euler angles, please leave the room) ... angle vectors or quaternion. We will you the first with $R\triangleq exp(w)$ and $w\in R^3$ and unconstrained 3d angle vector.

The problem can then be written:

Decide: 
- $w\in R^3$ the ellipsoid orientation
- $c \in R^3$ the ellipsoid center
- $r \in R^3$ the radii

Minimizing: the ellipsoid volum $\prod_{i=1}^{3} r_i$

Subject to:
- $r>=0$
- $\for v \in V:  (v-c)^T A (v-c) \le 1$
with $V$ the list of vertices, and $A$ defined as above mentionned from the decision variables.



### Helpers 
When writing this problem, $A$ and $c$ will be represented in the same frame as $V$. Remember that the vertices of the shape are given in the shape frame. Converting them in the joint frame can be done with geom.placement:

In [ ]:
geom.placement.act(vertices[0,:])

We will need the following simple Casadi helper to get rid of the SX/MX syntax.

In [ ]:
# %load tp4/generated/encapsulating_ellipses_helper
cw = casadi.SX.sym('w',3)
exp = casadi.Function('exp3',[cw], [cpin.exp3(cw)])


Now write an optimization problem with Casadi to compute the representation of the minimal encapsulating ellipsoid in the frame of the parent joint. For that, follow the steps:

1. Define the decision variables $w,c,r$.

In [ ]:
# %load tp4/generated/encapsulating_ellipses_vars
opti = casadi.Opti()
var_w = opti.variable(3)
var_r = opti.variable(3)
var_c = opti.variable(3)


2. Shape the matrix $A$ from the decision variables.

In [ ]:
# %load tp4/generated/encapsulating_ellipses_RA
# The ellipsoid matrix is represented by w=log3(R),diag(P) with R,P=eig(A)
R = exp(var_w)
A = R@casadi.diag(1/var_r**2)@R.T


3. Define the cost.

In [ ]:
# %load tp4/generated/encapsulating_ellipses_cost
totalcost = var_r[0]*var_r[1]*var_r[2]


4. Define the radius positivity.

In [ ]:
# %load tp4/generated/encapsulating_ellipses_rplus
opti.subject_to( var_r >= 0)


5. Define the encapsulating constraint.

In [ ]:
# %load tp4/generated/encapsulating_ellipses_points
for g_v in vertices:
    # g_v is the vertex v expressed in the geometry frame.
    # Convert point from geometry frame to joint frame
    j_v = geom.placement.act(g_v)
    # Constraint the ellipsoid to be including the point
    opti.subject_to( (j_v-var_c).T@A@(j_v-var_c)  <= 1  )


6. Solve and recover the optimal values.

In [ ]:
# %load tp4/generated/encapsulating_ellipses_solve
opti.minimize(totalcost)
opti.solver("ipopt") # set numerical backend
opti.set_initial(var_r,10)

sol = opti.solve_limited()

sol_r = opti.value(var_r)
sol_A = opti.value(A)
sol_c = opti.value(var_c)
sol_R = opti.value(exp(var_w))


### Display and future use

The ellipoid can be displayed in Meshcat. You need to give the radii to create the shape, and then place it at the center with proper orientaion. So Meshcat needs $(c,r,R)$ and not $(A,c)$. You can get $r,R$ from $A$ by Eigen decomposion.

In [ ]:
# %load tp4/generated/encapsulating_ellipses_meshcat
# Build the ellipsoid 3d shape
# Ellipsoid in meshcat
viz.addEllipsoid('el',sol_r,[.3,.9,.3,.3])
# jMel is the placement of the ellipsoid in the joint frame
jMel = pin.SE3(sol_R,sol_c)


We can now place the initial shape geom, the vertices represented by small spheres, and the ellispoid, at a consistent location.

In [ ]:
# %load tp4/generated/encapsulating_ellipses_vizplace
# Place the body, the vertices and the ellispod at a random configuration oMj_rand
oMj_rand = pin.SE3.Random()
viz.applyConfiguration(viz.getViewerNodeName(geom,pin.VISUAL),oMj_rand)
for i in  np.arange(0,vertices.shape[0]):
    viz.applyConfiguration(f'world/point_{i}',
                           oMj_rand.act(vertices[i]).tolist()+[1,0,0,0])
viz.applyConfiguration('el',oMj_rand*jMel)


## Inverse geometry with collisions

We will now write an inverse geometry problem under noncollision constraint. Ideally, the noncollision constraint should be that the distance between the bodies is positive (we might even argue whether this is sufficient or not). HPP-FCL is computing this function, yet differenciating it is more difficult as distances between nonstrictly-convex object is not a smooth function. Recent works, for example the work of Louis Montaut (2022), give some direction to solve this issue, but they are not yet fully available, and in particular not with the automatic differentiation system of Casadi.

We are then going to use a simple collision constraint: keep a point outside of an ellispoid by making the quadratic ellispoid equation larger than 1:
$$ \forall \mathcal{E}, \forall p, (^\mathcal{E}p-c_\mathcal{E})^T A_\mathcal{E} (^\mathcal{E}p-c_\mathcal{E}) \ge 1$$

with $\mathcal{E} = (A_\mathcal{E},c_\mathcal{E})$ the ellispoid en $^\mathcal{E}p$ the representation of the obstacle $p$ (a simple point) in the ellispoid frame.

Louis Montaut (2022). Collision Detection Accelerated: An Optimization Perspective, by L Montaut, Q Le Lidec, V Petrik, J Sivic, J Carpentier. RSS


We are going to do that for a UR-10 robot positioning its end effector, for you to adapt it to any humanoid robot you want within a full optimal control setting.

In [ ]:
# %load tp4/generated/invgeom_with_obstacles_hyper
### HYPER PARAMETERS
Mtarget = pin.SE3(pin.utils.rotate('y', 3), np.array([-0.8, -0.1, 0.2]))  # x,y,z
q0 = np.array([ 0,5,3,0,2,0 ])
endEffectorFrameName = 'tool0'


In [ ]:
# %load tp4/generated/invgeom_with_obstacles_model
# The pinocchio model is what we are really interested by.
model = robot.model
data = model.createData()
endEffector_ID = model.getFrameId(endEffectorFrameName)


### UR-10 ellipsoids and obstacles
You can compute the encapsulating ellispoids with the first part of the notebook, for the robot you prefer. Do that for all meaningfull bodies of your robot, and store them in a list which each items e contains the following fields:
- e.A: the ellipsoid matrix
- e.center: its center
- e.rotation: the orientation of its main axes
- e.radius: the 3 radii
- e.placement: SE3(e.rotation,e.center)
- e.id: the id of the joint it is attached to
- e.name: for display

In case you need it, here is the UR10 set up, using the SimpleNamespace class.

In [ ]:
from types import SimpleNamespace

In [ ]:
# %load tp4/generated/invgeom_with_obstacles_ellipses
# These values are computed using encapsulating_ellipse.py
ellipses = [
    SimpleNamespace(name="shoulder_lift_joint",
                    A=np.array([[75.09157846,  0.34008563, -0.08817025],
                                [ 0.34008563, 60.94969446, -0.55672959],
                                [-0.08817025, -0.55672959,  3.54456814]]),
                    center=np.array([-1.05980885e-04, -5.23471160e-02,  2.26280651e-01])),
    SimpleNamespace(name="elbow_joint",
                    A=np.array([[ 1.30344372e+02, -5.60880392e-02, -1.87555288e-02],
                                [-5.60880392e-02,  9.06119040e+01,  1.65531606e-01],
                                [-1.87555288e-02,  1.65531606e-01,  4.08568387e+00]]),
                    center=np.array([-2.01944435e-05,  7.22262249e-03,  2.38805264e-01])),
    SimpleNamespace(name="wrist_1_joint",
                    A=np.array([[ 2.31625634e+02,  5.29558437e-01, -1.62729657e-01],
                                [ 5.29558437e-01,  2.18145143e+02, -1.42425434e+01],
                                [-1.62729657e-01, -1.42425434e+01,  1.73855962e+02]]),
                    center=np.array([-9.78431524e-05,  1.10181763e-01,  6.67932259e-03])),
    SimpleNamespace(name="wrist_2_joint",
                    A=np.array([[ 2.32274519e+02,  1.10812959e-01, -1.12998357e-02],
                                [ 1.10812959e-01,  1.72324444e+02, -1.40077876e+01],
                                [-1.12998357e-02, -1.40077876e+01,  2.19132854e+02]]),
                    center=np.array([-2.64650554e-06,  6.27960760e-03,  1.11112087e-01])),
]


In [ ]:
# %load tp4/generated/invgeom_with_obstacles_ellipses_2
for e in ellipses:
    e.id = robot.model.getJointId(e.name)
    l,P = np.linalg.eig(e.A)
    e.radius = 1/l**.5
    e.rotation = P
    e.placement = pin.SE3(P,e.center)


Obstacles are stored in a similar list, with each item o containing:
- o.name: for display
- o.radius: for visualizing it as a sphere in meshcat
- o.pos: the 3d position in world frame.

In [ ]:
# %load tp4/generated/invgeom_with_obstacles_obstacles
# Obstacle positions are arbitrary. Their radius is meaningless, just for visualization.
obstacles = [
    SimpleNamespace(radius=.01, pos=np.array([-.4,0.2+s,0.5]),name=f"obs_{i_s}")
    for i_s,s in enumerate(np.arange(-.5,.5,.1))
]


Here is the visualisation of the set up.

In [ ]:
# %load tp4/generated/invgeom_with_obstacles_vizellipses
for e in ellipses:
    viz.addEllipsoid(f'el_{e.name}',e.radius,[.3,.9,.3,.3])
for io,o in enumerate(obstacles):
    viz.addSphere(f'obs_{io}',o.radius,[.8,.3,.3,.9])


In [ ]:
# %load tp4/generated/invgeom_with_obstacles_vizsimples
# --- Add box to represent target
# Add a vizualization for the target
boxID = "world/box"
viz.addBox(boxID, [.05, .1, .2], [1., .2, .2, .5])
# Add a vizualisation for the tip of the arm.
tipID = "world/blue"
viz.addBox(tipID, [.08] * 3, [.2, .2, 1., .5])


In [ ]:
import time

In [ ]:
# %load tp4/generated/invgeom_with_obstacles_viz
def displayScene(q,dt=1e-1):
    '''
    Given the robot configuration, display:
    - the robot
    - a box representing endEffector_ID
    - a box representing Mtarget
    '''
    pin.framesForwardKinematics(model,data,q)
    M = data.oMf[endEffector_ID]
    viz.applyConfiguration(boxID, Mtarget)
    viz.applyConfiguration(tipID, M)
    for e in ellipses:
        M = data.oMi[e.id]
        viz.applyConfiguration(f'el_{e.name}',M*e.placement)
    for io,o in enumerate(obstacles):
        viz.applyConfiguration(f'obs_{io}',pin.SE3(np.eye(3),o.pos))
    viz.display(q)
    time.sleep(dt)
displayScene(robot.q0)


### Writing the problem
The inverse geometry with collision will be written:

Decide: $q \in \mathcal{Q}$ the robot configuration

Minimizing: $|| tool(q) - target ||^2$

Subject to: $$\forall \mathcal{E}, \forall p, (^\mathcal{E}p-c_\mathcal{E})^T A_\mathcal{E} (^\mathcal{E}p-c_\mathcal{E}) \ge 1$$

with $tool(q)$ the end effector position with respect to a given $target$, $\mathcal{E}$ the ellipsoids described in associated joint frames, and $^\mathcal{E}p = \ ^oM_\mathcal{E}^{-1} \ ^op$ the position of each obstacle expressed in the joint frames associated to each ellispoid.

Classically, we need the basic Casadi helpers to get read of the SX/MX formulation.


In [ ]:
# %load tp4/generated/invgeom_with_obstacles_casadi
# --- Casadi helpers
cmodel = cpin.Model(model)
cdata = cmodel.createData()

cq = casadi.SX.sym("q",model.nq,1)
cpin.framesForwardKinematics(cmodel,cdata,cq)
error6_tool = casadi.Function('etool', [cq],
                             [ cpin.log6(cdata.oMf[endEffector_ID].inverse() * cpin.SE3(Mtarget)).vector ])
error3_tool = casadi.Function('etool', [cq],
                             [ cdata.oMf[endEffector_ID].translation - Mtarget.translation ])
error_tool = error3_tool


For each ellispoid, we need to compute the obstacle position in local frame.

In [ ]:
# %load tp4/generated/invgeom_with_obstacles_e_pos
cpos = casadi.SX.sym('p',3)
for e in ellipses:
    # Position of the obstacle cpos in the ellipse frame.
    e.e_pos =  casadi.Function(f"e{e.id}",[cq,cpos],
                                 [ cdata.oMi[e.id].inverse().act(casadi.SX(cpos)) ])
                


Using these helpers, write the optimization problem following these steps.

1. Declare the variables, and compute the cost.

In [ ]:
# %load tp4/generated/invgeom_with_obstacles_opti
opti = casadi.Opti()
var_q = opti.variable(model.nq)
totalcost = casadi.sumsqr(error_tool(var_q))


2. For each pair ellispoid-obstacle, add a constraint.

In [ ]:
# %load tp4/generated/invgeom_with_obstacles_constraint
for e in ellipses:
    for o in obstacles:
        # obstacle position in ellipsoid (joint) frame
        e_pos = e.e_pos(var_q,o.pos)
        opti.subject_to( (e_pos-e.center).T@e.A@(e_pos-e.center) >=1 )


3. Solve

In [ ]:
# %load tp4/generated/invgeom_with_obstacles_solve
opti.minimize(totalcost)
p_opts = dict(print_time=False, verbose=False)
s_opts = dict(print_level=0)
opti.solver("ipopt") # set numerical backend
opti.set_initial(var_q,robot.q0)

# Caution: in case the solver does not converge, we are picking the candidate values
# at the last iteration in opti.debug, and they are NO guarantee of what they mean.
try:
    sol = opti.solve_limited()
    sol_q = opti.value(var_q)
except:
    print('ERROR in convergence, plotting debug info.')
    sol_q = opti.debug.value(var_q)
